# Fine-Tuning the CNN on Cancer and Subtype





In [1]:
import os
import sys
from pathlib import Path

import pandas as p
import numpy as np

import tensorflow as tf
from tensorflow import keras
#Add the parent directory to access ENV variables
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#Import of necessary paths ( GDC data Path and Dataset folder)
from config import THYROID_PATH, MODEL_PATH, RESULTS_PATH

2025-04-11 15:30:20.717354: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-11 15:30:20.768920: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-11 15:30:20.768968: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-11 15:30:20.768978: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-11 15:30:20.776997: I tensorflow/core/platform/cpu_feature_g

In [2]:
FinalPath = Path(THYROID_PATH,'FinalData.npy')
npzfiles = np.load(FinalPath,allow_pickle=True)


#output_folder = Path(RESULTS_PATH,'UnfilteredCancer/')
#output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [3]:
def get_uncompiled_model(reset_last_layer=False):
    
    model = keras.models.load_model(os.path.join(MODEL_PATH,'pan-cancer-leaky-relu'))
    if(reset_last_layer):
        output_follicolar= keras.layers.Dense(1, activation='sigmoid',name='output_follicolar')(model.layers[-2].output)
        model = keras.models.Model(inputs=model.input, outputs = [output_follicolar])
    return model

def get_compiled_model(metrics=None,reset_last_layer=False):
    
    if(metrics is None):
        metrics = [
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'), 
              keras.metrics.BinaryAccuracy(name='accuracy'),
              keras.metrics.Precision(name='precision'),
              keras.metrics.Recall(name='recall'),
              keras.metrics.AUC(name='auc'),
              keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        ]
        
    model = get_uncompiled_model(reset_last_layer)
    model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(3e-5),metrics=metrics)
    
    return model

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

def make_or_restore_model(checkpoint_dir = "ckpt", reset_last_layer=False):
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model(reset_last_layer=reset_last_layer)

In [4]:
#Retrain Base Model for thyroid cancer classification

In [5]:
X_train = np.nan_to_num(npzfiles['X_cancer_train'])
y_train = npzfiles['y_cancer_train']

)

model = make_or_restore_model('ckpt-thyroid-cancer')

cancer_history = model.fit(X_train,y_train, batch_size=8, epochs=30, callbacks=early_stopping_cb, validation_split=0.2, shuffle=True)

cancer_df = pd.DataFrame(cancer_history.history)
cancer_df.to_csv("fit_history/thyroid-cancer.csv")
model.save(os.path.join(MODEL_PATH,"thyroid-cancer"))

Creating a new model


2025-04-11 15:30:32.351490: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-11 15:30:32.395337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-04-11 15:30:32.395592: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/30


2025-04-11 15:30:45.382553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2025-04-11 15:30:48.744583: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7efcdc292660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-04-11 15:30:48.744670: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 5000, Compute Capability 7.5
2025-04-11 15:30:48.750794: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-11 15:30:48.888382: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


42/42 [==============================] - 42s 629ms/step - loss: nan - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 30.0000 - fn: 299.0000 - accuracy: 0.0912 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5000 - prc: 0.9088 - val_loss: nan - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 14.0000 - val_fn: 69.0000 - val_accuracy: 0.1687 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.8313
Epoch 2/30
42/42 [==============================] - 24s 575ms/step - loss: nan - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 30.0000 - fn: 299.0000 - accuracy: 0.0912 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5000 - prc: 0.9088 - val_loss: nan - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 14.0000 - val_fn: 69.0000 - val_accuracy: 0.1687 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_prc: 0.8313
Epoch 3/30
42/42 [==============================] - 24s 577ms/step - loss: nan - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 30.0000 - fn

NameError: name 'pd' is not defined

In [ ]:
X_train = npzfiles['X_subtype_train']
y_train = npzfiles['y_subtype_train']

model = make_or_restore_model('ckpt-thyroid-subtype', reset_last_layer=True)

subtype_history = model.fit(X_train,y_train, batch_size=8, epochs=30, callbacks=early_stopping_cb, validation_split=0.2, shuffle=True)
subtype_df = pd.DataFrame(subtype_history.history)
subtype_df.to_csv(os.path.join('fit_history','thyrois-subtype.csv'))

model.save(os.path.join(MODEL_PATH,"thyroid-subtype"))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.2,shuffle=True, stratify=y)

from collections import Counter
X_train_fill = SimpleImputer(strategy='constant').fit_transform(X_train,y_train)
X_train_over,y_train_over = over.fit_resample(X_train_fill,y_train)
X_train_under,y_train_under = under.fit_resample(X_train_over,y_train_over)

X_test_fill = SimpleImputer(strategy='constant').fit_transform(X_test,y_test)
X_test_over,y_test_over = over.fit_resample(X_test_fill,y_test)
X_test_under,y_test_under = under.fit_resample(X_test_over,y_test_over)

print(Counter(y_train))
print(Counter(y_train_under))

print(Counter(y_test))
print(Counter(y_test_under))

